In [1]:
from common import *

import gzip
import os
import shutil
import sqlite3
from urllib.parse import urlparse

import duckdb
import pandas as pd
import requests
from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import yaml
import json


In [2]:
print("verify output is being rendered")

verify output is being rendered


In [3]:
# Initialize cache dictionaries for predict_from_normalized_env_packages
# todo how to move the definitions for function that use these globals? Or just use caching around the function?
ancestor_cache = {}
descendant_cache = {}

In [4]:
# todo deal with circularity in env package prediction -> env triad reporting

# todo this on-demand NCBI curie extraction and annotation recapitulates work that is being added to
# https://portal.nersc.gov/project/m3408/biosamples_duckdb/
# via 
#   although that doesn't detect auto-incremented curies from  spreadsheet dragging

# todo if more caching is desired, it should probably take the form of saving dataframes for TSV

# eventually, dig up a complete JSON gold biosample dump for non-hybrid biosample counts

# overall run time (if NCBI biosamples and goldData are cached): ~ 10 minutes

# todo make it clearer whether biosamples or studies are being counted
# count nmdc or gold studies too?

# Isolated Task Settings
**Pre-assembled settings block below are preferred**

_For making a Soil env_broad_scale voting sheet vs a Sediment env_local_scale sheet, etc._

todo: bundle these into dicts so they don't have to be modified independently and kept in sync with one another.

In [ ]:
output_file_name = "voting_sheets_output/sediment_env_local_scale_voting_sheet.tsv"

In [ ]:
# semantic_anchor = 'ENVO:00000428' # biome for env_broad_scale
semantic_anchor = 'ENVO:01000813' # astronomical body part "abp" for env_local_scale
# semantic_anchor = 'ENVO:00010483' # environmental material for env_medium

## context selectors

In [ ]:
gold_context_selectors =  [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]


In [ ]:
# ncbi_context_selector = 'env_broad_scale'
ncbi_context_selector = 'env_local_scale'
# ncbi_context_selector = 'env_medium'

In [ ]:
# nmdc_context_selector= 'env_broad_scale_id'
nmdc_context_selector= 'env_local_scale_id'
# nmdc_context_selector= 'env_medium_id'

## package aka environment aka extension selectors

In [ ]:
# plant_first_where = "s1.value like 'host-associated > plants%'"
# sediment_first_where = "lower(s1.value) like 'environmental > aquatic%sediment%'"
# soil_first_where = "s1.value like 'environmental > terrestrial > soil%'"
# water_first_where = "s1.value like 'environmental > aquatic%' and lower(s1.value) not like '%sediment%'"

plant_first_where = "lower(s1.value) like '%plant%'" # picks up waste water treatment plant
sediment_first_where = "lower(s1.value) like '%sediment%'"
soil_first_where = "lower(s1.value) like '%soil%'"
water_first_where = "lower(s1.value) like '%aquatic%' and lower(s1.value) not like '%sediment%'"

In [ ]:
gold_first_where = sediment_first_where

In [ ]:
# todo new since soil: why are we only considering MIMS.me for discovering appropriate env triad values?
#   there's usually a roughly equal number of biosamples from in each extension for MIMS.me and 

# ncbi_package_selector = 'plant-associated.6.0'
ncbi_package_selector = 'sediment.6.0'
# ncbi_package_selector = 'soil.6.0'
# ncbi_package_selector = 'water.6.0'

In [ ]:
# nmdc_package_selector = 'plant-associated'
nmdc_package_selector = 'sediment'
# nmdc_package_selector = 'soil'
# nmdc_package_selector = 'water'


In [ ]:
GOLDTERMS_NA = '' # ???

GOLDTERMS_PLANT_ASSOCIATED = GOLDTERMS_NA # host associated -> viridiplantae? take a string approach!
GOLDTERMS_SEDIMENT = 'GOLDTERMS:3985' #  doesn't have any subclasses
GOLDTERMS_SOIL = 'GOLDTERMS:4212'
GOLDTERMS_WATER = 'GOLDTERMS:3984'

# GOLDTERMS:4180, 'Environmental > Aquatic > Freshwater > Pond > Sediment' and ~64 more don't share a common root
# poetry run runoak -i sqlite:obo:goldterms info 't~sediment'


In [ ]:
goldterms_root = GOLDTERMS_NA

## selecting name and version of one enum for comparison

In [ ]:
# only the Soil enums have legacy definitions (v10.7 and earlier?)

# CONTEXT_ENUM = "EnvBroadScaleSoilEnum"
CONTEXT_ENUM = "EnvLocalScaleSoilEnum"
# CONTEXT_ENUM = "EnvMediumSoilEnum"

# CONTEXT_ENUM = ""

In [ ]:
# todo: add columns for membership in multiple enums from multiple version of the schema?
#  like sediment local vs soil local and water local (once that's completed)
#  get them from schema files or something prior to that? sems like the voting sheets are too raw/preliminary for that
#   can use a more recent schema url for more recent enums!

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

# previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

In [ ]:
# todo: don't call the column "legacy_pv". use the name of the enum and the version of the schema?

comparison_enum_column_name = 'EnvLocalSoilEnum_10_7'
# comparison_enum_column_name = 'EnvLocalScaleSoilEnum_11_1'
# comparison_enum_column_name = 'no_comparison_enum'

# Pre-compiled settings
Do not "run all cells below" from any one of these pre-compiled blocks. 
Run one configuration block, and then proceed to "Additional Settings"

## plant associated env_broad_scale

In [ ]:
output_file_name = "voting_sheets_output/plant_associated_env_broad_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00000428' # biome for env_broad_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_broad_scale'

nmdc_context_selector= 'env_broad_scale_id'

## package aka environment aka extension selectors
plant_first_where = "lower(s1.value) like '%plant%'"  # picks up waste water treatment plant
gold_first_where = plant_first_where

ncbi_package_selector = 'plant-associated.6.0'

nmdc_package_selector = 'plant-associated'

CONTEXT_ENUM = "EnvBroadScaleSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvBroadScaleSoilEnum_11_1'


## sediment env_broad_scale

In [ ]:
output_file_name = "voting_sheets_output/sediment_env_broad_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00000428' # biome for env_broad_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_broad_scale'

nmdc_context_selector= 'env_broad_scale_id'

## package aka environment aka extension selectors
sediment_first_where = "lower(s1.value) like '%sediment%'"
gold_first_where = sediment_first_where

ncbi_package_selector = 'sediment.6.0'

nmdc_package_selector = 'sediment'

CONTEXT_ENUM = "EnvBroadScaleSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvBroadScaleSoilEnum_11_1'


## soil env_broad_scale

In [ ]:
output_file_name = "voting_sheets_output/soil_env_broad_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00000428' # biome for env_broad_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_broad_scale'

nmdc_context_selector= 'env_broad_scale_id'

## package aka environment aka extension selectors
soil_first_where = "lower(s1.value) like '%soil%'"
gold_first_where = soil_first_where

ncbi_package_selector = 'soil.6.0'

nmdc_package_selector = 'soil'

CONTEXT_ENUM = "EnvBroadScaleSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvBroadScaleSoilEnum_10_7'


## plant associated env_local_scale

In [ ]:
output_file_name = "voting_sheets_output/plant_associated_env_local_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:01000813' # astronomical body part "abp" for env_local_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_local_scale'

nmdc_context_selector= 'env_local_scale_id'

## package aka environment aka extension selectors
plant_first_where = "lower(s1.value) like '%plant%'"  # picks up waste water treatment plant
gold_first_where = plant_first_where

ncbi_package_selector = 'plant-associated.6.0'

nmdc_package_selector = 'plant-associated'

CONTEXT_ENUM = "EnvLocalScaleSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvLocalScaleSoilEnum_11_1'


## sediment env_local_scale

In [ ]:
output_file_name = "voting_sheets_output/sediment_env_local_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:01000813' # astronomical body part "abp" for env_local_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_local_scale'

nmdc_context_selector= 'env_local_scale_id'

## package aka environment aka extension selectors
sediment_first_where = "lower(s1.value) like '%sediment%'"
gold_first_where = sediment_first_where

ncbi_package_selector = 'sediment.6.0'

nmdc_package_selector = 'sediment'

CONTEXT_ENUM = "EnvLocalScaleSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvLocalScaleSoilEnum_11_1'


## soil env_local_scale

In [ ]:
output_file_name = "voting_sheets_output/soil_env_local_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:01000813' # astronomical body part "abp" for env_local_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_local_scale'

nmdc_context_selector= 'env_local_scale_id'

## package aka environment aka extension selectors
soil_first_where = "lower(s1.value) like '%soil%'"
gold_first_where = soil_first_where

ncbi_package_selector = 'soil.6.0'

nmdc_package_selector = 'soil'

CONTEXT_ENUM = "EnvLocalScaleSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvLocalScaleSoilEnum_10_7'


## plant associated env_medium

In [ ]:
output_file_name = "voting_sheets_output/plant_associated_env_medium_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00010483' # environmental material for env_medium

## context selectors
gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_medium'

nmdc_context_selector= 'env_medium_id'

## package aka environment aka extension selectors
plant_first_where = "lower(s1.value) like '%plant%'"  # picks up waste water treatment plant
gold_first_where = plant_first_where

ncbi_package_selector = 'plant-associated.6.0'

nmdc_package_selector = 'plant-associated'

CONTEXT_ENUM = "EnvMediumSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvMediumSoilEnum_11_1'


## sediment env_medium

In [ ]:
output_file_name = "voting_sheets_output/sediment_env_medium_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00010483' # environmental material for env_medium

## context selectors
gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_medium'

nmdc_context_selector= 'env_medium_id'

## package aka environment aka extension selectors
sediment_first_where = "lower(s1.value) like '%sediment%'"
gold_first_where = sediment_first_where

ncbi_package_selector = 'sediment.6.0'

nmdc_package_selector = 'sediment'

CONTEXT_ENUM = "EnvMediumSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvMediumSoilEnum_11_1'


## soil env_medium

In [ ]:
output_file_name = "voting_sheets_output/soil_env_medium_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00010483' # environmental material for env_medium

## context selectors
gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_medium'

nmdc_context_selector= 'env_medium_id'

## package aka environment aka extension selectors
soil_first_where = "lower(s1.value) like '%soil%'"
gold_first_where = soil_first_where

ncbi_package_selector = 'soil.6.0'

nmdc_package_selector = 'soil'

CONTEXT_ENUM = "EnvMediumSoilEnum"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'EnvMediumSoilEnum_10_7'


# water env_broad_scale

In [ ]:
output_file_name = "voting_sheets_output/water_env_broad_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00000428' # biome for env_broad_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_broad_scale'

nmdc_context_selector= 'env_broad_scale_id'

## package aka environment aka extension selectors
water_first_where = "lower(s1.value) like '%aquatic%' and lower(s1.value) not like '%sediment%'"
gold_first_where = water_first_where

ncbi_package_selector = 'water.6.0'

nmdc_package_selector = 'water'

CONTEXT_ENUM = ""

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'no_comparison_enum'



## water env_local_scale

In [5]:
output_file_name = "voting_sheets_output/water_env_local_scale_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:01000813' # astronomical body part "abp" for env_local_scale

gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_local_scale'

nmdc_context_selector= 'env_local_scale_id'

## package aka environment aka extension selectors
water_first_where = "lower(s1.value) like '%aquatic%' and lower(s1.value) not like '%sediment%'"
gold_first_where = water_first_where

ncbi_package_selector = 'water.6.0'

nmdc_package_selector = 'water'

CONTEXT_ENUM = ""

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'no_comparison_enum'

## water env_medium

In [ ]:
output_file_name = "voting_sheets_output/water_env_medium_voting_sheet.tsv"

# bootstrapping
semantic_anchor = 'ENVO:00010483' # environmental material for env_medium

## context selectors
gold_context_selectors = [
    'mixs:env_broad',
    'mixs:env_local',
    'mixs:env_medium'
]

ncbi_context_selector = 'env_medium'

nmdc_context_selector= 'env_medium_id'

## package aka environment aka extension selectors
water_first_where = "lower(s1.value) like '%aquatic%' and lower(s1.value) not like '%sediment%'"
gold_first_where = water_first_where

ncbi_package_selector = 'water.6.0'

nmdc_package_selector = 'water'

CONTEXT_ENUM = ""

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

comparison_enum_column_name = 'no_comparison_enum'

# Additional Settings
safe to run all of these cells sequentially

In [6]:
# Approved prefixes (case-insensitive)
approved_prefixes = ['ENVO']

In [7]:
MIN_ANNOTATION_LEN = 3

In [8]:
NMDC_RUNTIME_BASE_URL = 'https://api.microbiomedata.org/nmdcschema/'
STUDY_SET_COLLECTION = 'study_set'
BIOSAMPLE_SET_COLLECTION = 'biosample_set'

In [9]:
envo_adapter_string = "sqlite:obo:envo"

In [10]:
env_package_override_file = 'mam-env-package-overrides.tsv'
override_column = 'mam_inferred_env_package'

In [11]:
# ncbi_duckdb_url = 'https://portal.nersc.gov/project/m3408/biosamples_duckdb/ncbi_biosamples_2024-09-23.duckdb.gz'
ncbi_duckdb_url = 'https://portal.nersc.gov/project/m3408/biosamples_duckdb/ncbi_biosamples.duckdb.gz'

In [12]:
gold_data_url = "https://gold.jgi.doe.gov/download?mode=site_excel"
gold_data_file_name = "goldData.xlsx" # goldData.xlsx: Microsoft Excel 2007+
gold_csv_file_name = "gold_biosamples.csv"
BIOSAMPLES_SHEET = "Biosample"

In [13]:
goldterms_semsql_url = "https://s3.amazonaws.com/bbop-sqlite/goldterms.db.gz"

In [14]:
all_nmdc_biosamples_file = 'all_nmdc_biosamples.json'

# CURIe Constants

In [15]:
BIOME = 'ENVO:00000428'
TERRESTRIAL_BIOME = 'ENVO:00000446'
AQUATIC_BIOME = 'ENVO:00002030'
ABP = 'ENVO:01000813'
ENVIRONMENTAL_SYSTEM = 'ENVO:01000254'
ENVIRONMENTAL_MATERIAL = 'ENVO:00010483'

SOIL = 'ENVO:00001998'
LIQUID_WATER = 'ENVO:00002006'
WATER_ICE = 'ENVO:01000277'

HUMAN_CONSTRUCTION = 'ENVO:00000070'
BUILDING = 'ENVO:00000073'
BUILDING_PART = 'ENVO:01000420'

# Settings-based Queries

In [16]:
# todo could this have been done with a OAK query, eliminating the need to explicitly download the file?

goldterms_envo_query = f"""
SELECT
	*
FROM
	statements s
WHERE
	predicate in ('{"', '".join(gold_context_selectors)}')"""

In [17]:
ncbi_biosamples_per_annotation_query = f"""
SELECT content, COUNT(1) AS count 
FROM attributes 
WHERE harmonized_name = '{ncbi_context_selector}' AND package_content like '%{ncbi_package_selector}'
GROUP BY content
ORDER BY COUNT(1) DESC
"""

In [18]:
ncbi_bioprojects_per_annotation_query = f"""
SELECT
	a.content,
	count(DISTINCT l.content) AS count
FROM
	main.ATTRIBUTES a
JOIN main.links l 
	ON
	a.id = l.id
WHERE
	l.target = 'bioproject'
	AND harmonized_name = 'env_local_scale'
	AND package_content LIKE '%soil.6.0'
GROUP BY
	a.content
ORDER BY
	count(DISTINCT l.content) DESC ;
"""

In [19]:
ncbi_X_per_annotation_query = ncbi_bioprojects_per_annotation_query

In [20]:
# and s1.subject = s1.stanza eliminates matches on blank node annotation rows (probably wouldn't change results but adds a little overhead)

extension_query = f"""
select
		s1.subject ,
		s2.predicate,
		COALESCE (s2."object",
	s2."value") as content
from
	statements s1
join statements s2 on 
	s1.subject = s2.subject
where
	{gold_first_where}
	and s1.predicate = 'rdfs:label'
	and s1.subject = s1.stanza
	and s2.predicate in ('mixs:env_broad', 'mixs:env_local', 'mixs:env_medium', 'mixs:mixs_extension', 'rdfs:label', 'mixs:other', 'mixs:anatomical_site', 'mixs:host_taxon') ;
"""


# Locally Defined Functions
_Currently using locally-defined cache dictionaries_

In [21]:
def predict_from_normalized_env_packages(df_raw, adapter):
    # Apply the function to the relevant columns

    df = df_raw.copy()

    print(df.shape)
    for column in ['env_broad_scale_id', 'env_local_scale_id', 'env_medium_id']:
        df[f'{column}_ancestors'] = df[column].apply(lambda x: get_hierarchy_terms(x, adapter)['ancestors'])
        df[f'{column}_descendants'] = df[column].apply(lambda x: get_hierarchy_terms(x, adapter)['descendants'])

    # Vectorize each set of terms separately
    broad_scale_ancestors = vectorize_terms(df, 'env_broad_scale_id_ancestors')
    broad_scale_descendants = vectorize_terms(df, 'env_broad_scale_id_descendants')

    local_scale_ancestors = vectorize_terms(df, 'env_local_scale_id_ancestors')
    local_scale_descendants = vectorize_terms(df, 'env_local_scale_id_descendants')

    medium_ancestors = vectorize_terms(df, 'env_medium_id_ancestors')
    medium_descendants = vectorize_terms(df, 'env_medium_id_descendants')

    # Combine all feature matrices
    X = hstack([
        broad_scale_ancestors,
        broad_scale_descendants,
        local_scale_ancestors,
        local_scale_descendants,
        medium_ancestors,
        medium_descendants
    ])

    # Filter the DataFrame to only include non-null rows for the target column
    df_filtered = df[df['normalized_env_package'].notnull() & (df['normalized_env_package'] != "")]

    # Extract the target variable
    y = df_filtered['normalized_env_package']

    # Ensure X corresponds to the filtered rows
    X_filtered = X[df_filtered.index]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_filtered, y, test_size=0.3, random_state=42)

    # Train a Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = clf.predict(X_test)

    # Evaluate the model
    print(classification_report(y_test, y_pred))

    # not determining confidence for each class nor saving any diagnostics any more

    return clf.predict(X)

In [22]:
def get_hierarchy_terms(my_curie: str, adapter) -> dict:
    """
    Extract ancestor and descendant terms from the ontology for a given CURIE,
    using caching to improve performance and filtering by 'is_a' relationships.

    Args:
        my_curie (str): CURIE identifier for the ontology term.
        adapter: Ontology adapter.

    Returns:
        dict: Dictionary containing lists of ancestor and descendant terms.
    """
    if my_curie not in ancestor_cache:
        try:
            ancestors = list(adapter.ancestors(my_curie, predicates=[IS_A]))
            ancestor_cache[my_curie] = [adapter.label(ancestor) for ancestor in ancestors if ancestor]
        except Exception as my_e:
            print(f"Error retrieving ancestors for {my_curie}: {my_e}")
            ancestor_cache[my_curie] = []

    if my_curie not in descendant_cache:
        try:
            descendants = list(adapter.descendants(my_curie, predicates=[IS_A]))
            descendant_cache[my_curie] = [adapter.label(descendant) for descendant in descendants if descendant]
        except Exception as my_e:
            print(f"Error retrieving descendants for {my_curie}: {my_e}")
            descendant_cache[my_curie] = []

    return {
        'ancestors': ancestor_cache[my_curie],
        'descendants': descendant_cache[my_curie],
    }

# Procedural Code Starts Here

In [23]:
# Determine the filenames and target directory for the NCBI DuckDB
ncbi_compressed_filename = urlparse(ncbi_duckdb_url).path.split('/')[-1]
ncbi_filename = os.path.splitext(ncbi_compressed_filename)[0]
ncbi_compressed_file_path = os.path.join(ncbi_compressed_filename)
ncbi_uncompressed_file_path = os.path.join(ncbi_filename)


In [24]:
if os.path.isfile(ncbi_uncompressed_file_path):
    print(f"{ncbi_uncompressed_file_path} is already present in the current working directory.")
else:
    if os.path.isfile(ncbi_compressed_file_path):
        print(f"{ncbi_compressed_file_path} is already present in the current working directory.")
    else:
        print(f"{ncbi_compressed_file_path} needs to be downloaded")
        ncbi_response = requests.get(ncbi_duckdb_url)
        with open(ncbi_compressed_file_path, "wb") as f:
            f.write(ncbi_response.content)
        # ~ 2 minutes @ 250 Mbps
    
    # Unzip the compressed file and save the extracted file in target directory
    print(f"{ncbi_compressed_file_path} needs to be unpacked")
    with gzip.open(ncbi_compressed_file_path, "rb") as f_in:
        with open(ncbi_uncompressed_file_path, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    # ~ 2 minutes

ncbi_biosamples.duckdb is already present in the current working directory.


In [25]:
ncbi_conn = duckdb.connect(database=ncbi_uncompressed_file_path, read_only=True)

In [26]:
envo_adapter = get_adapter(envo_adapter_string)

# Anchor aka bootstrapping classes

In [27]:
anchor_descendants = get_curie_descendants_label_dict(semantic_anchor, [IS_A], envo_adapter)

In [28]:
anchor_descendants_lod = curie_descendants_label_dict_to_lod(anchor_descendants)

In [29]:
anchor_descendants_frame = curie_descendants_label_lod_to_df(anchor_descendants_lod)

In [30]:
anchor_descendants_frame

,curie,label
0,ENVO:00001999,marine water body
1,ENVO:01000188,tropical savanna biome
2,ENVO:00000487,paternoster lake
3,ENVO:01000860,temperate marine upwelling biome
4,ENVO:01000199,mediterranean forest biome
...,...,...
1731,ENVO:01000429,burrow
1732,ENVO:01000431,mixed forest
1733,ENVO:01000536,factory
1734,ENVO:00000873,freshwater biome


# Classes from the reference enumeration

In [31]:
sv = get_schemaview_from_source(previous_submission_schema_url)

In [32]:
# todo break out slow steps into its own cell

try:
    CONTEXT_ENUM_def = sv.get_enum(CONTEXT_ENUM)
    context_pvs_keys = list(CONTEXT_ENUM_def.permissible_values.keys())
except AttributeError as e:
    # Handle the AttributeError
    print(f"An AttributeError occurred: {e}")
    context_pvs_keys =[]
    

An AttributeError occurred: 'NoneType' object has no attribute 'permissible_values'


In [33]:
print(context_pvs_keys)

[]


In [34]:
initially_parsed_context_pvs = parse_hierarchically_underscored_strings(context_pvs_keys)

In [35]:
deduped_context_pvs = dedupe_underscoreless_pvs(initially_parsed_context_pvs)

In [36]:
pv_validation_results = validate_curie_label_list_dict(deduped_context_pvs, envo_adapter, print_flag=True)

In [37]:
pv_validation_results

{'problems': [], 'valids': []}

# Get the CURIEs used in NMDC Biosample annotations

In [38]:
if os.path.isfile(all_nmdc_biosamples_file):
    print(f"{all_nmdc_biosamples_file} is present in the current working directory and will be read into all_nmdc_biosamples.")
    # with open(all_nmdc_biosamples_file, 'r') as file:
    #     all_nmdc_biosamples = yaml.full_load(file)
    # read as json
    with open(all_nmdc_biosamples_file, 'r') as f:
        all_nmdc_biosamples = json.load(f)

else:
    print(f"All NMDC Biosamples need to be fetched and saved to {all_nmdc_biosamples_file}")
    all_nmdc_biosamples = get_docs_from_nmdc_collection(NMDC_RUNTIME_BASE_URL,
                                               BIOSAMPLE_SET_COLLECTION)
    # with open(all_nmdc_biosamples_file, 'w') as file:
    #     documents = yaml.dump(all_nmdc_biosamples, file)
    # save as json
    with open(all_nmdc_biosamples_file, 'w') as f:
        json.dump(all_nmdc_biosamples, f)

# this saves network traffic. could use JSON for faster performance. 
# 1 minute for network fetch and JSON write?!
# 1 minute for yaml read
# instantaneous for JSON read?

All NMDC Biosamples need to be fetched and saved to all_nmdc_biosamples.json
Fetched page 1 with 1000 documents. Total fetched: 1000
Fetched page 2 with 1000 documents. Total fetched: 2000
Fetched page 3 with 1000 documents. Total fetched: 3000
Fetched page 4 with 1000 documents. Total fetched: 4000
Fetched page 5 with 1000 documents. Total fetched: 5000
Fetched page 6 with 1000 documents. Total fetched: 6000
Fetched page 7 with 1000 documents. Total fetched: 7000
Fetched page 8 with 1000 documents. Total fetched: 8000
Fetched page 9 with 362 documents. Total fetched: 8362
All documents fetched.


## Inference of env_package annotations 

In [39]:
env_pacakge_overrides = tsv_to_dict_of_dicts(env_package_override_file, 'id')

In [40]:
# todo show env_pacakge_overrides as a data frame
#   with some other columns for context?

In [41]:
biosample_contexts_lod = biosamples_lod_context_extractor(all_nmdc_biosamples, envo_adapter,
                                                          my_env_pacakge_overrides=env_pacakge_overrides)

# ~ 10 seconds, lots of logging

Overriding env_package for biosample nmdc:bsm-11-002vgm56 from soil to soil
Overriding env_package for biosample nmdc:bsm-11-006pnx90 from plant-associated to plant-associated
Overriding env_package for biosample nmdc:bsm-11-00dkyf35 from soil to soil
Overriding env_package for biosample nmdc:bsm-11-00hrxp98 from  to None
Overriding env_package for biosample nmdc:bsm-11-00m15h97 from  to None
Overriding env_package for biosample nmdc:bsm-11-00yhef97 from  to None
Overriding env_package for biosample nmdc:bsm-11-011z7z70 from  to None
Overriding env_package for biosample nmdc:bsm-11-0169zs66 from  to None
Overriding env_package for biosample nmdc:bsm-11-01bbrr08 from  to None
Overriding env_package for biosample nmdc:bsm-11-01f6m423 from  to None
Overriding env_package for biosample nmdc:bsm-11-01g9wf51 from  to None
Overriding env_package for biosample nmdc:bsm-11-01jah904 from  to None
Overriding env_package for biosample nmdc:bsm-11-01teww33 from  to None
Overriding env_package for b

In [42]:
nmdc_biosample_contexts_frame = pd.DataFrame(biosample_contexts_lod)

In [43]:
nmdc_biosample_contexts_frame.to_csv('nmdc_biosample_asserted_and_normalized_env_package.tsv', sep='\t', index=False)

In [44]:
# print a value count for the normalized_env_package column
print("Value counts for normalized_env_package column:")
print(nmdc_biosample_contexts_frame['normalized_env_package'].value_counts(dropna=False))

Value counts for normalized_env_package column:
normalized_env_package
None                                               5673
soil                                               1707
plant-associated                                    401
water                                               192
sediment                                            165
miscellaneous natural or artificial environment     140
host-associated                                      61
hydrocarbon resources-fluids_swabs                   23
Name: count, dtype: int64


In [45]:
package_predictions = predict_from_normalized_env_packages(nmdc_biosample_contexts_frame, envo_adapter)

# these predictions often have a f1 of 1.00
# many people might find that hard to believe

(8362, 14)
                                                 precision    recall  f1-score   support

                                host-associated       1.00      1.00      1.00        28
             hydrocarbon resources-fluids_swabs       1.00      0.86      0.92         7
miscellaneous natural or artificial environment       1.00      1.00      1.00        51
                               plant-associated       1.00      1.00      1.00       106
                                       sediment       1.00      1.00      1.00        46
                                           soil       1.00      1.00      1.00       518
                                          water       0.98      1.00      0.99        51

                                       accuracy                           1.00       807
                                      macro avg       1.00      0.98      0.99       807
                                   weighted avg       1.00      1.00      1.00       807



In [46]:
nmdc_biosample_contexts_frame['predicted_env_package'] = package_predictions

In [47]:
nmdc_biosample_contexts_frame.to_csv('nmdc_biosample_asserted_normalized_and_inferred_env_package.tsv', sep='\t', index=False)

## env-package inference complete

To-do: save this and don't recreate it if it's available

Then get it reviewed by other NMDC stakeholders and inject it into MongoDB if approved

## Destructively filter `nmdc_biosample_contexts_frame` by `env_package` 

In [ ]:
nmdc_biosample_contexts_frame.shape

In [ ]:
nmdc_biosample_contexts_frame = nmdc_biosample_contexts_frame[
    nmdc_biosample_contexts_frame['predicted_env_package'] == nmdc_package_selector]

In [ ]:
nmdc_biosample_contexts_frame.shape

# Long process of inferring OBO foundry CURIes from NCBI Biosamples

## Start by getting unique annotations? Pre-counted by Biosamples

Current task is to provide counts by "study" aka Bioproject in addition to Biosample counts or instead of Biosamples counts if necessary

In [ ]:
ncbi_frame = ncbi_conn.execute(ncbi_X_per_annotation_query).fetchdf()


In [ ]:
ncbi_frame.insert(0, 'serial_number', range(1, len(ncbi_frame) + 1))

In [ ]:
# includes values with counts of one... useful for discovering drag-down submissions?

## MIxS and NCBI guidelines imply environmental context slots are multivalued
and that the pipe `|` should be used as a delimiter

there's an envo_count value below that indicates how ofter other delimiteres might be used

In [ ]:
ncbi_frame['content_list'] = ncbi_frame['content'].str.split('|')

In [ ]:
# todo is there any reason to not do this ?
ncbi_frame = ncbi_frame[ncbi_frame['content'].notna() & (ncbi_frame['content'] != '')]

In [ ]:
ncbi_frame['content_count'] = ncbi_frame['content_list'].apply(len)

In [ ]:
ncbi_frame.shape

In [ ]:
ncbi_frame = ncbi_frame.explode('content_list').reset_index(drop=True)

In [ ]:
ncbi_frame.shape

## splitting adds ~ 5% more rows
which might be important since were currently using a longest annotation strategy here

In [ ]:
# how many content_list strings contain envo multiple times now?

In [ ]:
ncbi_frame['envo_count'] = ncbi_frame['content_list'].str.lower().str.count("envo")

In [ ]:
ncbi_frame['envo_count'].value_counts()

## If my math is correct, about 0.1% of the annotations still contain multiple CURIes 
after splitting on pipes

There will also be annotations with multiple label-like strings that weren't split because they weren't delimited on pipes
That might be a source of lost information since we are using a longest-match annotator here
I.e. there could be annotations with multiple hits worth keeping

## Parsing out CURIEs

this has a few limitations. The function only tries pre-specified prefixes (['ENOV'] by default) and only considers colons and underscores valid delimiters.

In [ ]:
ncbi_frame[['extracted_label', 'extracted_curie']] = ncbi_frame['content_list'].apply(parse_curie_label)

In [ ]:
parse_failures = ncbi_frame[
    (ncbi_frame['envo_count'] > 0) & (ncbi_frame['extracted_curie'].isna() | (ncbi_frame['extracted_curie'] == ''))]


## In what kinds of cases could no CURIe be parsed
despite the presence of "ENVO" in the content string?

In [ ]:
parse_failures

## Retrieve the labels for the parsed CURIes

In [ ]:
ncbi_frame['real_label'] = ncbi_frame['extracted_curie'].apply(envo_adapter.label)

## Apply oaklib annotation to the strings after CURIe removal
Actually the annotator can (sometimes?) detect colon-delimited CURIEs with lower case prefixes

This returns CURIes with evidence but not necessarily the label corresponding to the CURIe

In [ ]:
# Apply the annotation function to each row in the 'label' column
ncbi_frame['longest_annotation_curie'] = ncbi_frame['extracted_label'].apply(
    lambda x: get_longest_annotation_curie(x, envo_adapter, MIN_ANNOTATION_LEN))

# this cell only takes ~ 1 minute, but generates a lot of "ERRORS" and WARNINGS in a red font
#   while loading the ontologies that are used for annotating


## Add the labels for the CURIes identified though oaklib annotation of strings

In [ ]:
ncbi_frame['longest_annotation_label'] = ncbi_frame['longest_annotation_curie'].apply(envo_adapter.label)

In [ ]:
ncbi_frame

## we now have a list of CURIes for each normalized annotation

This could be because the submitter provided a CURIe and a label that don't match
*One* case of this is dragging a CURIe down a column in a spreadsheet, expecting it to be copied,
but actually auto-incrementing it

Now attempt to find one best CURIe for each annotation... by now we have lost the ability to retain multiple legitimate
but improperly separated CURIes

In [ ]:
# todo don't accept extracted curie if no real label?
# any kind of string similarity checking for label of annotated curie vs extracted label ?
# look for long stretches of curies?
# can we measure the beneficial impact of any of this? current crux: how to distribute counts

ncbi_frame['curie_list'] = ncbi_frame.apply(
    lambda my_row: list({my_row['extracted_curie'], my_row['longest_annotation_curie']} - {None}),
    axis=1
)

ncbi_frame['unique_curie_count'] = ncbi_frame['curie_list'].apply(len)

In [ ]:
ncbi_frame['unique_curie_count'].value_counts()

In [ ]:
double_curie_frame = ncbi_frame[ncbi_frame['unique_curie_count'] > 1]

In [ ]:
double_curie_frame = double_curie_frame[['extracted_curie', 'longest_annotation_curie']]

In [ ]:
double_curie_frame = double_curie_frame.drop_duplicates()

In [ ]:
double_curie_frame[['extracted_prefix', 'extracted_local_id']] = double_curie_frame['extracted_curie'].str.split(':', expand=True)

In [ ]:
double_curie_frame['extracted_local_id_int'] = pd.to_numeric(double_curie_frame['extracted_local_id'], errors='coerce').astype('Int64')

In [ ]:
double_curie_frame

In [ ]:
# Ensure extracted_local_id_int is unique and sorted
unique_sorted_series = double_curie_frame['extracted_local_id_int'].dropna().drop_duplicates().sort_values()


In [ ]:
# Find stretches
stretches_dict = find_consecutive_stretches_dict(unique_sorted_series)


In [ ]:
# Convert the stretches dictionary into a DataFrame
stretches_df = stretches_dict_to_long_dataframe(stretches_dict)

`stretches_df` shows groups of extracted EnvoO ids (CURIes without prefix or padding zeros) that share a common CURIe by oaklib annotation of the textual part. This may not be the best or only way to address these spurious drag-stretch, auto-incremented CURIes

Ie 1001458 corresponds to ENVO:01001458, 'mist'

_although it theoretically could have been ENVO:1001458 since EnvO CURIes can have 7 or 8 digits_

In group 9, there are another ~ 50 sequential id values, all corresponding to environmental context annotations whose best oak-annotated class is ENVO:01001803, 'tropical forest'!

How much of an impact does this have? 

In [ ]:
stretches_df

In [ ]:
# Perform the left merge
double_curie_frame = double_curie_frame.merge(
    stretches_df,
    left_on='extracted_local_id_int',
    right_on='value',
    how='left'
)

In [ ]:
stretch_summary_df = summarize_stretch_groups(double_curie_frame)


For stretch 9, which included extracted CURIes from ENVO:01001458 to ENVO:01001511, the oaklib test annotation of 100% of the submitted environmental context annotations was ENVO:01001803, so we will keep that and disregard all of the CURIes from the stretch


In [ ]:
stretch_summary_df

In [ ]:
decisive_fraction_threshold = 0.9

In [ ]:
decisive_stretch_summary_df = stretch_summary_df[stretch_summary_df['fraction'] >= decisive_fraction_threshold]

In [ ]:
decisive_stretch_summary_df

In [ ]:
# Perform the left merge
double_curie_frame = double_curie_frame.merge(
    decisive_stretch_summary_df,
    left_on='stretch_id',
    right_on='stretch_id',
    how='left'
)

In [ ]:
double_curie_frame

In [ ]:
drag_evidence_frame = double_curie_frame[double_curie_frame['stretch_id'] >= 1]
drag_evidence_frame = drag_evidence_frame[['extracted_curie', 'longest_annotation_curie']]
drag_evidence_frame['drag_evidence'] = True

In [ ]:
drag_evidence_frame

In [ ]:
ncbi_frame = ncbi_frame.merge(
    drag_evidence_frame,
    left_on=['extracted_curie', 'longest_annotation_curie'],
    right_on=['extracted_curie', 'longest_annotation_curie'],
    how='left'
)

In [ ]:
# Initialize dragless_curie_list with curie_list values
ncbi_frame["dragless_curie_list"] = ncbi_frame["curie_list"]

# Update dragless_curie_list based on the condition
for index, row in ncbi_frame.iterrows():
    if row["drag_evidence"] is True:
        if row["longest_annotation_curie"] is not None:
            ncbi_frame.at[index, "dragless_curie_list"] = [row["longest_annotation_curie"]]
        else:
            ncbi_frame.at[index, "dragless_curie_list"] = []

ncbi_frame['dragless_curie_count'] = ncbi_frame['dragless_curie_list'].apply(len)

In [ ]:
ncbi_frame['unique_curie_count'].value_counts()

In [ ]:
ncbi_frame['dragless_curie_count'].value_counts()

## The extent of multiple detected CURIes has been reduced ~ 4.5 fold 
(for soil env_local_scale)

Isolate the submitter annotations for which there's clearly one best CURIe after removing the drag-stretches

In [ ]:
ncbi_frame.shape

In [ ]:
ncbi_frame_undisputed = ncbi_frame[ncbi_frame['dragless_curie_count'] <= 1]

In [ ]:
ncbi_frame_undisputed.shape

In [ ]:
ncbi_frame_disputed = ncbi_frame[ncbi_frame['dragless_curie_count'] > 1]

In [ ]:
ncbi_frame_disputed.shape

In [ ]:
ncbi_frame_disputed = ncbi_frame_disputed.explode("dragless_curie_list", ignore_index=True)


In [ ]:
ncbi_frame_disputed.shape

In [ ]:
ncbi_frame_disputed["dragless_curie_list"] = ncbi_frame_disputed["dragless_curie_list"].apply(lambda x: [x])

## Just include all of the remaining disputed CURIe assingments

In [ ]:
# Combine the rows of ncbi_frame_undisputed and ncbi_frame_disputed into a new DataFrame
ncbi_disputes_exploded_frame = pd.concat([ncbi_frame_undisputed, ncbi_frame_disputed], ignore_index=True)


In [ ]:
ncbi_disputes_exploded_frame.shape

In [ ]:
ncbi_disputes_exploded_frame

In [ ]:
ncbi_disputes_exploded_frame['post_explode_curie_count'] = ncbi_disputes_exploded_frame['dragless_curie_list'].apply(len)

In [ ]:
ncbi_disputes_exploded_frame['post_explode_curie_count'].value_counts()

In [ ]:
# Set 'post_explode_curie' to the 0th item in 'dragless_curie_list'
ncbi_disputes_exploded_frame["post_explode_curie"] = ncbi_disputes_exploded_frame["dragless_curie_list"].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
)

In [ ]:

ncbi_biosample_scoped_counts = (
    ncbi_disputes_exploded_frame.groupby("post_explode_curie")["count"].sum().reset_index()
)

ncbi_biosample_scoped_counts.columns = ['curie', 'ncbi_scoped_count']

This is currently a count of Biosamples for which the indicated CURIes can be extracted or inferred by oaklib annotation, after removal of drag-stretch, auto-incremented CURIes

In [ ]:
ncbi_biosample_scoped_counts

## GOLD mappings/Biosample counts hybrid

we're currently including
- mappings only

and have retired the previous use of
- mappings in hybrid with biosample counts

And we're casting a wide net, especially for the hybrid approach
- searching for 'soil', 'sediment' etc. in GOLDTERMS labels without anchoring them like 'Environmental > Aquatic > Sediment'
- retrieving the CURIes for env_broad_scale, env_local_scale and env_medium for all voting sheets, and trusting orthogonal filtering to remove the inappropriate CURIes

Should we now add (or switch to) direct biosample counts of GOLD "envo" annotations?

Efficient retrieval of  all GOLD data in a given scope isn't easy

In [ ]:
# Determine the filenames and target directory
goldterms_compressed_filename = urlparse(goldterms_semsql_url).path.split('/')[-1]
goldterms_filename = os.path.splitext(goldterms_compressed_filename)[0]
target_dir = os.path.join("..", "..")  # Two levels up

# Print to confirm the filenames
print(goldterms_filename)

In [ ]:
# Fetch the contents from the URL and save compressed file in target directory
goldterms_response = requests.get(goldterms_semsql_url)
goldterms_compressed_file_path = os.path.join(target_dir, goldterms_compressed_filename)
with open(goldterms_compressed_file_path, "wb") as f:
    f.write(goldterms_response.content)

# ~ 1 second

In [ ]:
# Unzip the compressed file and save the extracted file in target directory
goldterms_uncompressed_file_path = os.path.join(target_dir, goldterms_filename)
with gzip.open(goldterms_compressed_file_path, "rb") as f_in:
    with open(goldterms_uncompressed_file_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

# ~ 1 second

In [ ]:
## that's all fast. don't bother caching

In [ ]:
goldterms_conn = sqlite3.connect(goldterms_uncompressed_file_path)

# GOLDTERMS only approach

In [ ]:
goldterms_result = pd.read_sql_query(extension_query, goldterms_conn)

In [ ]:
goldterms_result

In [ ]:
# # todo: save this kind of content before subsetting on an environment
# #   the subsetting is currently baked into the query
# 
# # see also goldterms_queries.ipynb in MAM's Collab
# goldterms_result.to_csv("goldterms_single_environment_mappings_long.tsv", sep="\t", index=False)

In [ ]:
goldterms_only_curies = goldterms_result.loc[goldterms_result['predicate'].isin(gold_context_selectors), 'content']


In [ ]:
goldterms_only_curies = goldterms_only_curies.unique().tolist()

In [ ]:
# goldterms_only_curies

# Make lists of CURIEs
which will determine
- the rows in the table
- the boolean filter columns

In [ ]:
anchor_curies = list(anchor_descendants_frame['curie'])
legacy_pv_curies = [i['curie'] for i in pv_validation_results['valids']]

biome_curies = list(envo_adapter.descendants(BIOME, predicates=[IS_A])) # 
terrestrial_biome_curies = list(envo_adapter.descendants(TERRESTRIAL_BIOME, predicates=[IS_A]))
aquatic_biome_curies = list(envo_adapter.descendants(AQUATIC_BIOME, predicates=[IS_A]))
abp_curies = list(envo_adapter.descendants(ABP, predicates=[IS_A]))
env_sys_curies = list(envo_adapter.descendants(ENVIRONMENTAL_SYSTEM, predicates=[IS_A]))
env_mat_curies = list(envo_adapter.descendants(ENVIRONMENTAL_MATERIAL, predicates=[IS_A]))
obsoletes_curies = list(envo_adapter.obsoletes())

soil_curies = list(envo_adapter.descendants(SOIL, predicates=[IS_A])) # 
liquid_water_curies = list(envo_adapter.descendants(LIQUID_WATER, predicates=[IS_A])) # 
water_ice_curies = list(envo_adapter.descendants(WATER_ICE, predicates=[IS_A])) # 

human_construction_curies = list(envo_adapter.descendants(HUMAN_CONSTRUCTION, predicates=[IS_A])) #
building_curies = list(envo_adapter.descendants(BUILDING, predicates=[IS_A])) #
building_part_curies = list(envo_adapter.descendants(BUILDING_PART, predicates=[IS_A])) #


## Bootstrap the rows

In [ ]:
include_in_rows = set()

In [ ]:
include_in_rows.update(anchor_curies)

In [ ]:
include_in_rows.update(legacy_pv_curies)

In [ ]:
include_in_rows.update(nmdc_biosample_contexts_frame[nmdc_context_selector])

In [ ]:
include_in_rows.update(ncbi_frame['extracted_curie'])

In [ ]:
include_in_rows.update(ncbi_frame['longest_annotation_curie'])

In [ ]:
# include_in_rows.update(gold_env_filtered_biosamples_with_inferred['object'])

In [ ]:
include_in_rows.update(goldterms_only_curies)

In [ ]:
rows_lod = []

# Voting sheet rows and boolean columns

In [ ]:
for curie in include_in_rows:
    if curie is None:
        continue
        
    # ONCE AGAIN, assuming that EnvO is the only ontology we'll check against
    current_ancestors = list(envo_adapter.ancestors(curie, predicates=[IS_A])) # vs legacy_pv_curies
    ancestors_in_enum_count = len(set(current_ancestors) & set(legacy_pv_curies))
    
    current_descendants  = list(envo_adapter.descendants(curie, predicates=[IS_A])) # vs legacy_pv_curies
    descendants_in_enum_count  = len(set(current_descendants) & set(legacy_pv_curies))
    
    
    row = {
        'curie': curie,
        'label': envo_adapter.label(curie),
        'envo_native': False,
        'obsolete': False,
        comparison_enum_column_name: False,
        'ancestors_in_enum_count': ancestors_in_enum_count,
        'descendants_in_enum_count': descendants_in_enum_count,
        'abp': False,
        'env_sys': False,
        'biome': False,
        'terrestrial_biome': False,
        'aquatic_biome': False,
        'env_mat': False,
        'soil': False,
        'liquid water': False,
        'water ice': False,
        'human_construction': False,
        'building': False,
        'building_part': False,
        'goldterms_mappings': False,
    }
        
    if curie in biome_curies:
        row['biome'] = True
    if curie in terrestrial_biome_curies:
        row['terrestrial_biome'] = True
    if curie in aquatic_biome_curies:
        row['aquatic_biome'] = True
    if curie in abp_curies:
        row['abp'] = True
    if curie in env_sys_curies:
        row['env_sys'] = True
    if curie in env_mat_curies:
        row['env_mat'] = True
    if curie in soil_curies:
        row['soil'] = True
    if curie in liquid_water_curies:
        row['liquid water'] = True
    if curie in water_ice_curies:
        row['water ice'] = True
    if curie in human_construction_curies:
        row['human_construction'] = True
    if curie in building_curies:
        row['building'] = True
    if curie in building_part_curies:
        row['building_part'] = True
    if curie in legacy_pv_curies:
        row[comparison_enum_column_name] = True
    if curie in obsoletes_curies:
        row['obsolete'] = True
    if curie in goldterms_only_curies:
        row['goldterms_mappings'] = True
        
    try:
        prefix, local_id = curie.split(':')
        if prefix and prefix == 'ENVO' and row['label'] is not None:
            row['envo_native'] = True
    except Exception as e:
        # Print the exception message
        print(f"An error occurred: {e} trying to split {curie}")

    rows_lod.append(row)

# 2 minutes


# ^ Voting sheet rows and boolean columns

In [ ]:
rows_frame = pd.DataFrame(rows_lod)

In [ ]:
rows_frame

# Merge in NMDC counts

In [ ]:
nmdc_biosample_scoped_counts = nmdc_biosample_contexts_frame[nmdc_context_selector].value_counts().reset_index()
nmdc_biosample_scoped_counts.columns = ['curie', 'nmdc_scoped_count']


In [ ]:
nmdc_biosample_scoped_counts

In [ ]:
# Perform the left merge
rows_frame = rows_frame.merge(
    nmdc_biosample_scoped_counts,
    left_on='curie',
    right_on='curie',
    how='left'
)

# Merge in GOLD hybrid counts

In [ ]:
# gold_env_filtered_biosamples_with_inferred

In [ ]:
# gold_biosample_scoped_counts = gold_env_filtered_biosamples_with_inferred['object'].value_counts().reset_index()
# gold_biosample_scoped_counts.columns = ['curie', 'gold_hybrid_count']

In [ ]:
# gold_biosample_scoped_counts

In [ ]:
# # Perform the left merge
# rows_frame = rows_frame.merge(
#     gold_biosample_scoped_counts,
#     left_on='curie',
#     right_on='curie',
#     how='left'
# )

In [ ]:
rows_frame

In [ ]:
# gold and ncbi counts are slightly trickier
# for gold: including mappings only, mappings in hybrid with biosample counts. 
#    Switch to direct biosample counts of GOLD "envo" annotations?
# ncbi: we have extracted curies and annotated curies

## Merge in NCBI counts

In [ ]:
# Perform the left merge
rows_frame = rows_frame.merge(
    ncbi_biosample_scoped_counts,
    left_on='curie',
    right_on='curie',
    how='left'
)

In [ ]:
# 990 rows in https://docs.google.com/spreadsheets/d/12WH3eduBq2qSTy9zVF3n7fyajn6ssLZL/edit?gid=546570706#gid=546570706

In [ ]:
rows_frame.to_csv(output_file_name, sep="\t", index=False)

In [ ]:
ncbi_conn.close()